Import Libraries and Dependancies

In [14]:
import glob
import os
import pandas as pd
import skimage.io
import skimage.color
from skimage.feature import hog
from skimage import exposure
from sklearn.model_selection import train_test_split
from sklearn import svm
from skimage.feature import graycomatrix,graycoprops
from skimage import io, color, exposure
import cv2
from skimage.measure import shannon_entropy
from scipy.stats import mode
import numpy as np

Load Test images (for future testing)

In [15]:
test_csv_data = pd.read_csv('C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/test.csv',index_col=None)
test_new=test_csv_data.sort_values(by="name")
test_image_folder = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/test"
test_image_data={}
count=0
count_fail=0
for index, row in test_new.iterrows():
  test_img_name=(row.iloc[0])+".jpg"
  test_img_path = os.path.join(test_image_folder, test_img_name)
  test_img_label=row.iloc[1]
  if os.path.exists(test_img_path):
    test_image = skimage.io.imread(test_img_path)
    test_image_data[test_img_path] = {'image': test_image, 'label': test_img_label}
    count+=1
  else:
      count_fail+=1

count,count_fail



(2139, 0)

Helper to return metrics

In [7]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
def get_metrics(labels, predictions):
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='macro')
    return accuracy, f1, recall, precision

Histogram of Gradients

In [5]:
for img_path,data in test_image_data.items():
  image=data["image"]
  gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  fd, hog_image = hog(gray_segmented_image, pixels_per_cell=(16, 16), visualize=True)
  hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
  fd = fd.reshape(1, -1)
  data["hog_features"]=fd

Gray Level Co-occurence Matrix (GLCM) with Mean Shift Filtering

In [9]:
for img_path,data in test_image_data.items():
  image=data["image"]
  #filtered_image = cv2.pyrMeanShiftFiltering(image, sp=15, sr=30)
  image = skimage.filters.unsharp_mask(image, radius=2, amount=5)
  image=image.astype(np.uint8)
  gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  GLCM = graycomatrix(gray_segmented_image, [1], [0])
  GLCM_Energy = graycoprops(GLCM, 'energy')[0,0]
  data['Energy'] = GLCM_Energy
  GLCM_corr = graycoprops(GLCM, 'correlation')[0,0]
  data['Corr'] = GLCM_corr
  GLCM_diss = graycoprops(GLCM, 'dissimilarity')[0,0]
  data['Diss_sim'] = GLCM_diss
  GLCM_hom = graycoprops(GLCM, 'homogeneity')[0,0]
  data['Homogen'] = GLCM_hom
  GLCM_contr = graycoprops(GLCM, 'contrast')[0,0]
  data['Contrast'] = GLCM_contr


HU Moments


In [12]:
for img_path,data in test_image_data.items():
    image=data["image"]
    image = skimage.filters.unsharp_mask(image, radius=2, amount=5)
    image=image.astype(np.uint8)
    gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(gray_segmented_image*255, 128, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(image)
    hu_moments= cv2.HuMoments(moments)
    data["Hu_1"]=hu_moments[0][0]
    data["Hu_2"]=hu_moments[1][0]
    data["Hu_3"]=hu_moments[2][0]
    data["Hu_4"]=hu_moments[3][0]
    data["Hu_5"]=hu_moments[4][0]
    data["Hu_6"]=hu_moments[5][0]
    data["Hu_7"]=hu_moments[6][0]
    

Statistical Features

In [13]:
from scipy.stats import skew, kurtosis
for img_path,data in test_image_data.items():
  image=data["image"]
  image = skimage.filters.unsharp_mask(image, radius=2, amount=5)
  data['mean_value']= np.mean(image)
  data['std_deviation'] = np.std(image)
  data['entropy_value'] = shannon_entropy(image)
  data["median"] = np.median(image)
  data["percentile_25"] = np.percentile(image, 25)
  data["percentile_75"] = np.percentile(image, 75)
  data["skewness"] = skew(image, axis=None)
  data["kurtosis"] = kurtosis(image, axis=None)


ORB

In [65]:
from sklearn.decomposition import PCA

for img_path, data in image_data.items():
    image = data["image"]
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)

    if descriptors is not None:
        n_components = min(descriptors.shape[0], descriptors.shape[1])
        pca = PCA(n_components=n_components)
        orb_pca = pca.fit_transform(descriptors)
        data["orb_pca"] = orb_pca

    else:
        data["orb_pca"] = np.array([])
    


In [68]:
data["orb_pca"].shape, n_components

((500, 32), 32)

Print for verification

In [56]:
# Check the first few elements in feature_vectors
for i, feature_vector in enumerate(feature_vectors[:5]):
    print(f"Feature Vector {i+1}: {feature_vector}")




Feature Vector 1: [0.017072267697617077, 0.9843788650268777, 5.560535408512719, 0.19501167354734786, 62.28937591731898, 165.27353541056314, 45.646744694582765, 7.449294529043555, 172.0, 137.0, 199.0, 0.0017867533423917348, 1.702525212414615e-07, 1.5468648502994796e-09, 2.542699264984661e-10, 1.3386206180641189e-19, -9.504639648095484e-14, -8.666268872749686e-20, 67.40281249999994, 20.402812500000007, -24.597187499999986, 101.4028125, 92.4028125, 56.4028125, 95.4028125, 104.4028125, -53.59718749999998, -61.59718749999999, -93.59718749999998, 56.4028125, -114.59718749999998, -113.59718749999998, 59.4028125, 74.4028125, 110.40281250000001, 32.40281250000001, 28.402812500000007, 56.4028125, -60.59718749999999, 64.4028125, -114.59718749999998, 123.40281250000001, -64.59718749999999, -29.59718749999999, 16.402812500000007, 108.40281250000001, 60.4028125, 82.40281250000001, 129.4028125, 12.402812500000008, 57.40281250000001, -26.597187499999986, 38.4028125, -26.597187499999986, -45.5971874999

Scale and Pre Process Training and Test data for Model 

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

feature_vectors = []

for img_path, data in test_image_data.items():
    glcm_features = [data['Energy'], data['Corr'], data['Diss_sim'], data['Homogen'], data['Contrast']]
    statistical_features = [data['mean_value'], data['std_deviation'], data['entropy_value'], data['median'], data['percentile_25'], data['percentile_75'], data['skewness'], data['kurtosis']]
    hu_features = [data["Hu_1"],data["Hu_2"],data["Hu_3"],data["Hu_4"],data["Hu_5"],data["Hu_6"],data["Hu_7"]]
    # orb_features = data["orb_pca"]
    # orb_features_flattened = orb_features.flatten() if len(orb_features.shape) > 1 else orb_features
    combined_features = np.concatenate([glcm_features, statistical_features, hu_features], axis=0)
    feature_vectors.append(combined_features)


In [83]:
len(glcm_features), len(statistical_features), len(hu_features), len(orb_features), len(combined_features)

(5, 8, 7, 500, 20)

In [84]:

labels = [data['label'] for img_path, data in test_image_data.items()]

X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Support Vector Machine

In [85]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel="rbf", random_state=42, C=30, gamma=0.1)

svm_classifier.fit(X_train_scaled, y_train)

accuracy = svm_classifier.score(X_test_scaled, y_test)
test_predictions = svm_classifier.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- SVM --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))



--- SVM --- 
Accuracy:  67 
F1:  61 
Recall:  61 
Precision:  65


In [86]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
cross_val_score(svm.SVC(kernel="rbf",C=20,gamma=0.1), X_train_scaled, y_train, cv=5, scoring="accuracy")

array([0.6568323 , 0.6874028 , 0.67496112, 0.66562986, 0.67496112])

In [87]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(),{"C":[1,5,10,15,20],"kernel":["rbf","linear","sigmoid"]},cv=5,return_train_score=False)
clf.fit(X_train_scaled,y_train)
clf.cv_results_
cldf=pd.DataFrame(clf.cv_results_)

In [89]:
cldf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.290736,0.007137,0.154212,0.017794,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.625776,0.646967,0.625194,0.615863,0.642302,0.631221,0.011598,5
1,0.360574,0.025012,0.033516,0.008345,1,linear,"{'C': 1, 'kernel': 'linear'}",0.597826,0.604977,0.600311,0.576983,0.597201,0.595459,0.009635,10
2,0.318551,0.014399,0.060613,0.006851,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.430124,0.416796,0.413686,0.412131,0.418351,0.418218,0.006347,11
3,0.277682,0.012379,0.127383,0.012352,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.636646,0.653188,0.651633,0.632970,0.662519,0.647391,0.010990,4
4,0.711618,0.051274,0.028431,0.010384,5,linear,"{'C': 5, 'kernel': 'linear'}",0.607143,0.595645,0.601866,0.581649,0.597201,0.596701,0.008528,9
5,0.266906,0.016792,0.049552,0.003265,5,sigmoid,"{'C': 5, 'kernel': 'sigmoid'}",0.400621,0.398134,0.409020,0.413686,0.393468,0.402986,0.007357,12
6,0.276548,0.017159,0.118429,0.005150,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.656832,0.660964,0.662519,0.653188,0.665630,0.659827,0.004365,3
7,1.060485,0.052616,0.034824,0.007455,10,linear,"{'C': 10, 'kernel': 'linear'}",0.608696,0.590980,0.597201,0.587869,0.603421,0.597633,0.007690,8
8,0.259294,0.014989,0.056074,0.008334,10,sigmoid,"{'C': 10, 'kernel': 'sigmoid'}",0.399068,0.396579,0.405910,0.371695,0.396579,0.393966,0.011648,15
9,0.320959,0.027122,0.120580,0.012175,15,rbf,"{'C': 15, 'kernel': 'rbf'}",0.658385,0.667185,0.668740,0.659409,0.667185,0.664181,0.004363,2


In [90]:
clf.best_score_,clf.best_params_,clf.best_estimator_

(0.6679109956241608, {'C': 20, 'kernel': 'rbf'}, SVC(C=20))

In [91]:
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=40,learning_rate=0.2, booster='gbtree', max_depth=3)
bst.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=40, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [41]:
from sklearn.model_selection import GridSearchCV
param_grid={'n_estimators':[10,20,30,40,50],'max_depth':[1,3,5,7],'learning_rate':[0.01,0.1,0.2],'booster':['gbtree','gblinear','dart']}
clf=GridSearchCV(XGBClassifier(),param_grid=param_grid,cv=5,return_train_score=False)
clf.fit(X_train_scaled,y_train)
clf.cv_results_
cldf=pd.DataFrame(clf.cv_results_)

c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:22:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:22:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:22:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.w

In [42]:
cldf
clf.best_score_,clf.best_params_,clf.best_estimator_

(0.6262463413927339,
 {'booster': 'gbtree',
  'learning_rate': 0.2,
  'max_depth': 5,
  'n_estimators': 40},
 XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.2, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=5, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=40, n_jobs=None,
               num_parallel_tree=None, objective='multi:softprob', ...))

In [92]:
X=bst.score(X_test_scaled, y_test)
test_predictions = bst.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- XGB --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))


--- XGB --- 
Accuracy:  65 
F1:  57 
Recall:  57 
Precision:  62


In [93]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(solver='svd')

lda.fit(X_train_scaled, y_train)
test_predictions = lda.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- LDA --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))


--- LDA --- 
Accuracy:  60 
F1:  51 
Recall:  52 
Precision:  52


In [94]:
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(150, 100, 50), activation='relu', solver='adam', max_iter=300, batch_size=200)
mlp_classifier.out_activation_ = 'softmax'

mlp_classifier.fit(X_train_scaled, y_train)
test_predictions = mlp_classifier.predict(X_test_scaled)

accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- MLP --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))

--- MLP --- 
Accuracy:  67 
F1:  61 
Recall:  61 
Precision:  62


In [95]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
scores = cross_val_score(clf,X_train_scaled,y_train, cv=5)
print(scores.mean())
print("Cross-Validation Mean Accuracy:", scores.mean())
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", accuracy)


0.5301570665455986
Cross-Validation Mean Accuracy: 0.5301570665455986
Test set accuracy: 0.5416149068322982


In [96]:

clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf,X_train_scaled ,y_train , cv=5)
print("Cross-Validation Mean Accuracy:", scores.mean())
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", accuracy)


Cross-Validation Mean Accuracy: 0.610701969610618
Test set accuracy: 0.6186335403726708


In [97]:

clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
scores = cross_val_score(clf,X_train_scaled , y_train, cv=5)
print("Cross-Validation Mean Accuracy:", scores.mean())
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", accuracy)

Cross-Validation Mean Accuracy: 0.632151792355322
Test set accuracy: 0.6372670807453417
